In [29]:
import pandas as pd
from plotly import graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import spacy
from nltk import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import langid
import nltk
from nltk.corpus import words
sns.set_theme()
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt') 

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\MSI\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MSI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MSI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
data_imdb = pd.read_csv('IMDB_sentiment.csv', sep=',', index_col=0)
data_filmaffinity = pd.read_csv('reviews_filmaffinity.csv', sep='\|\|',engine='python')

Revisando un posible desbalanceo de clases

In [3]:
sentiment_count = data_imdb.sentiment.value_counts()
colors = {'positive': 'green', 'negative': 'red'}


fig = go.Figure(
    data= [go.Bar(y=sentiment_count.values, x=sentiment_count.index, marker=dict(color=[colors[s] for s in sentiment_count.index]))],
)
fig.update_layout(title='Gráfico de barra para el conteo de los comentarios positivos y negativos',
                  width=700, 
                  height=400,
                xaxis=dict(title='Sentiment', linewidth=2), 
                yaxis=dict(title='Count',  linewidth=2),)

fig.show()

Normalización de minúsculas

In [4]:
data_imdb['review_es'] = data_imdb.review_es.str.lower()
data_imdb.review_es.head(5)

0    uno de los otros críticos ha mencionado que de...
1    una pequeña pequeña producción.la técnica de f...
2    pensé que esta era una manera maravillosa de p...
3    básicamente, hay una familia donde un niño peq...
4    el "amor en el tiempo" de petter mattei es una...
Name: review_es, dtype: object

Tokenizar palabras 

In [5]:
data_imdb['review_es'] = data_imdb.review_es.apply(lambda x: word_tokenize(x))
data_imdb.review_es.head(5)

0    [uno, de, los, otros, críticos, ha, mencionado...
1    [una, pequeña, pequeña, producción.la, técnica...
2    [pensé, que, esta, era, una, manera, maravillo...
3    [básicamente, ,, hay, una, familia, donde, un,...
4    [el, ``, amor, en, el, tiempo, '', de, petter,...
Name: review_es, dtype: object

Eliminación de números y/o alfanuméricos

In [6]:
def alf_nums(rows):
    cadena = []
    for word in rows:
        if (word is not word.isdigit()) or (word is not word.isalnum()):
            try:
                float(word)
            except ValueError:
                cadena.append(word)

    return cadena

Esta función nos ayudará a eliminar las palabras en inglés (al menos varias de ellas)

In [7]:
stopwords_espanol = set(stopwords.words('spanish'))

def eliminar_palabras_ingles_serie(serie_texto):
    
    def eliminar_palabras_ingles_lista(lista_tokens):
        idioma_lista, _ = langid.classify(' '.join(lista_tokens))
        if idioma_lista == 'en':
            lista_filtrada = [palabra for palabra in lista_tokens if palabra.lower() not in stopwords_espanol]
        else:
            lista_filtrada = lista_tokens
        return lista_filtrada
    
    serie_filtrada = serie_texto.apply(eliminar_palabras_ingles_lista)
    return serie_filtrada



In [8]:
stopwords_spanish = stopwords.words('spanish')
def clean_stopWords(rows):
    no_stops = []
    for word in rows:
        if word not in stopwords_spanish:
            no_stops.append(word)
    return no_stops

In [9]:
data_imdb['review_es'] = data_imdb['review_es'].apply(alf_nums)
data_imdb.review_es.head(5)

0    [uno, de, los, otros, críticos, ha, mencionado...
1    [una, pequeña, pequeña, producción.la, técnica...
2    [pensé, que, esta, era, una, manera, maravillo...
3    [básicamente, ,, hay, una, familia, donde, un,...
4    [el, ``, amor, en, el, tiempo, '', de, petter,...
Name: review_es, dtype: object

Limpieza de palabras frecuentes en textos, pero que no aporta información nueva o clave en ello (stopwords)

In [10]:
data_imdb['review_es'] = data_imdb['review_es'].apply(clean_stopWords)
data_imdb.review_es.head(5)

0    [críticos, mencionado, después, ver, solo, oz,...
1    [pequeña, pequeña, producción.la, técnica, fil...
2    [pensé, manera, maravillosa, pasar, tiempo, fi...
3    [básicamente, ,, familia, niño, pequeño, (, ja...
4    [``, amor, tiempo, '', petter, mattei, películ...
Name: review_es, dtype: object

Uso de stemmer para preprocesar el texto 

In [11]:
data_filmaffinity.head()

,film_name,gender,film_avg_rate,review_rate,review_title,review_text
0,Ocho apellidos vascos,Comedia,"6,0",3.0,OCHO APELLIDOS VASCOS...Y NINGÚN NOMBRE PROPIO,La mayor virtud de esta película es su existen...
1,Ocho apellidos vascos,Comedia,"6,0",2.0,El perro verde,"No soy un experto cinéfilo, pero pocas veces m..."
2,Ocho apellidos vascos,Comedia,"6,0",2.0,Si no eres de comer mierda... no te comas esta...,Si no eres un incondicional del humor estilo T...
3,Ocho apellidos vascos,Comedia,"6,0",2.0,Aida: The movie,"No sé qué está pasando, si la gente se deja ll..."
4,Ocho apellidos vascos,Comedia,"6,0",2.0,UN HOMBRE SOLO (Julio Iglesias 1987),"""Pero cuando amanece,y me quedo solo,siento en..."


En este dataframe tratamos de capturar los datos que necesitamos puesto que contiene comantarios neutros, y la mayoría de nuestra información es acerca de los comentarios positivos o negativos

In [12]:
data_filmaffinity = data_filmaffinity.iloc[data_filmaffinity.query('review_rate>=7 or review_rate<=4').index].loc[:,('review_text','review_rate')]

In [13]:
data_filmaffinity.review_rate = data_filmaffinity.review_rate.map(lambda row: 'negative' if row<=4.0  else  'positive')

In [14]:
sentiment_count_fil = data_filmaffinity.review_rate.value_counts()

colors = {'positive': 'green', 'negative': 'red'}
fig = go.Figure(
    data= [go.Bar(y=sentiment_count_fil.values, x=sentiment_count_fil.index, marker=dict(color=[colors[s] for s in sentiment_count_fil.index]))],
)
fig.update_layout(title='Gráfico de barra para el conteo de los comentarios positivos y negativos',
                  width=700, 
                  height=400,
                xaxis=dict(title='Sentiment', linewidth=2), 
                yaxis=dict(title='Count',  linewidth=2),)

fig.show()

Normalizar a minúsculas

In [15]:
data_filmaffinity.review_text = data_filmaffinity.review_text.str.lower() 

Tokenizar las oraciones

In [16]:
data_filmaffinity.review_text = data_filmaffinity.review_text.map(lambda x: word_tokenize(x)) 

Eliminación de alfa numéricos y números

In [17]:
data_filmaffinity.review_text = data_filmaffinity.review_text.apply(alf_nums) 

Eliminación de stopwords

In [18]:
data_filmaffinity.review_text = data_filmaffinity.review_text.apply(clean_stopWords) 

Cambio de nombre en las columnas

In [19]:
data_all_reviews = data_filmaffinity._append(data_imdb.rename(columns={'review_es':'review_text','sentiment':'review_rate'}))

In [20]:
data_all_reviews.reset_index(inplace=True, drop=True)

In [21]:
data_all_reviews

,review_text,review_rate
0,"[mayor, virtud, película, existencia.el, hecho...",negative
1,"[experto, cinéfilo, ,, pocas, veces, tan, jueg...",negative
2,"[si, incondicional, humor, estilo, tele, 5.si,...",negative
3,"[sé, pasando, ,, si, gente, deja, llevar, moda...",negative
4,"[``, amanece, ,, quedo, solo, ,, siento, fondo...",negative
...,...,...
56362,"[pensé, película, hizo, buen, trabajo, derecha...",positive
56363,"[mala, parcela, ,, mal, diálogo, ,, mala, actu...",negative
56364,"[católica, enseñada, escuelas, primarias, parr...",negative
56365,"[voy, tener, desacuerdo, comentario, anterior,...",negative


NOTA: Para este proyecto se decidió usar la lematización, con el fin de tratar de conservar/identificar contextos y palabras claves

In [22]:
stemmer = SnowballStemmer('spanish')

def stemmer_text(rows:list):
    stemized_text = []

    for word in rows:

        stemized_text.append(stemmer.stem(word))
    
    return stemized_text

In [23]:
lemmer = spacy.load('es_core_news_sm')

def lematizer_text(rows):
    
    doc = [word.lemma_ for word in lemmer(rows)]

    return doc

Eliminar palabras en inglés, al menos las más reconocibles


In [24]:
data_all_reviews['review_text'] = eliminar_palabras_ingles_serie(data_all_reviews['review_text'])

Como posteriormente los distintos corpus se habían tokenizado, necesitamos ahora "juntar" en una sola cadena cada corpues, esto debido a un procedimiento de rendimiento

In [25]:
data_all_reviews['review_text'] = data_all_reviews['review_text'].map(lambda doc:' '.join(doc))

In [26]:
text = pd.Series()
for text_init in range(0,60000,10000):
    
    text = text._append(data_all_reviews['review_text'][text_init:text_init+10000].apply(lematizer_text))

In [27]:
data_all_reviews['review_text'] = text

In [28]:
data_all_reviews.to_parquet('data_cleaned.parquet')